In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Read meta data (ใช้รูป crop)
metadata = pd.read_csv('../input/font-detection-dataset-crop/label_train.csv')
# add .png to make datagenerator  read file 
metadata['parentId']  = metadata['parentId'] + '.png'

In [ ]:
# map fontSize integer
unique_vals = metadata['fontSize'].unique()
metadata['fontSize_int'] = metadata['fontSize'].replace(to_replace=unique_vals,
           value= list(range(len(unique_vals))))

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import BatchNormalization , Dense , Dropout , GlobalAveragePooling2D

In [ ]:
def createEffNetB0(input_shape):
    model = EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=input_shape,)
    return model

In [ ]:
# declare backbone model
basemodel = createEffNetB0((105, 105, 3))
basemodel.trainable = True 

In [ ]:
def create_model(base_model):
    
    x =  BatchNormalization()(base_model.output)
    x =  GlobalAveragePooling2D()(x)
    
    # Font Family
    dense_family = Dense(1024,activation = 'relu')(x)
    dropout_family = Dropout(0.2)(dense_family)
    output_family = Dense(6,activation = 'softmax')(dropout_family)
 
    model = tf.keras.Model(inputs = base_model.input , outputs = output_family)
    
    return model

In [ ]:
model = create_model(basemodel)

In [ ]:
model.compile(optimizer =  tf.keras.optimizers.Adam(),
              # กำหนด loss แบบ list ตาม order ที่create model or use 1 for all
            loss = [tf.keras.losses.SparseCategoricalCrossentropy()],
            metrics = ['acc']
            )

In [ ]:
# Create DATA PIPELINE 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

DATA_PATH = '../input/font-detection-dataset-crop/'

# use preprocessing_function match with model
image_generator = ImageDataGenerator(preprocessing_function = tf.keras.applications.efficientnet.preprocess_input)
train_generator = image_generator.flow_from_dataframe(metadata[:-20000],# edit for metadata[:-10000] for real data
                                    directory = DATA_PATH + 'image_train',
                                    x_col ='parentId',
                                    class_mode = 'raw',
                                    y_col = 'fontSize_int',
                                    batch_size = 128,
                                    seed = 69,
                                    shuffle = True,
                                    target_size=(105,105)
                                    )
validation_generator = image_generator.flow_from_dataframe(metadata[-20000:],# edit for metadata[-10000:] for real data
                                    directory = DATA_PATH + 'image_train',
                                    x_col ='parentId',
                                    class_mode = 'raw',
                                    y_col = 'fontSize_int',
                                    batch_size = 128,
                                    seed = 69,
                                    shuffle = True,
                                    target_size=(105,105)
                                    )


In [ ]:
from tensorflow.keras.callbacks import Callback , EarlyStopping , ModelCheckpoint , ReduceLROnPlateau

checkpoint = ModelCheckpoint( filepath = './eff_size_best.h5', monitor='val_loss', verbose=0, save_best_only=True,
                            save_weights_only=True, mode='auto', save_freq='epoch')
earlystop =  EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0,
                            mode='auto', baseline=None, restore_best_weights=False)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='min')

callbacks = [checkpoint,earlystop,reduce_lr]

In [ ]:
model.load_weights('../input/fontsize-weight/eff_size_best.h5')

In [ ]:
history = model.fit(x=train_generator,
                    validation_data=validation_generator,
                    epochs=5,
                    callbacks=callbacks)